In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import time
csv_start_time = time.monotonic()    
data = pd.read_csv("./datasets/kddcup_data_10_percent_corrected.csv", header = None, nrows = None)
csv_end_time = time.monotonic()    
print("processing in: {} seconds".format(csv_end_time - csv_start_time))

#df_results = pd.read_csv("results_test/20190517-143649_fullRun.csv")

processing in: 1.8439999999827705 seconds


In [2]:
def fit_transform_cols(data, object_only = True):
    if object_only == True:
        object_data =  data.select_dtypes(include=['object']).copy()
    else:
        object_data =  data.copy()
    # print(types.head())
    output = pd.DataFrame(data).copy()
    le_list = {}
    for col in object_data:   
        #print(np.sort(object_data[col].unique()) )
        le = preprocessing.LabelEncoder()
        le.fit(object_data[col])    
        le_list[col] = le
        #print("classes of {} are:{}".format(col,np.sort(le.classes_)))        
        output[col] = le.transform(object_data[col])
    return output, le_list

In [3]:
labels = (data.iloc[:,41]) 
le = preprocessing.LabelEncoder()
le.fit(labels)
print("classes of labels are:{}".format(le.classes_))
labels_encoded = le.transform(labels)
data_encoded, le_list = fit_transform_cols(data)

X_train, X_test, y_train, y_test = train_test_split(data_encoded, labels_encoded, test_size=0.80, random_state=42)
print(X_test.head())


classes of labels are:['back.' 'buffer_overflow.' 'ftp_write.' 'guess_passwd.' 'imap.' 'ipsweep.'
 'land.' 'loadmodule.' 'multihop.' 'neptune.' 'nmap.' 'normal.' 'perl.'
 'phf.' 'pod.' 'portsweep.' 'rootkit.' 'satan.' 'smurf.' 'spy.' 'teardrop.'
 'warezclient.' 'warezmaster.']
        0   1   2   3     4     5   6   7   8   9   ...   32    33    34  \
317921   0   0  14   9  1032     0   0   0   0   0  ...  255  1.00  0.00   
171422   0   0  14   9  1032     0   0   0   0   0  ...  255  1.00  0.00   
312181   0   0  14   9  1032     0   0   0   0   0  ...  255  1.00  0.00   
87346    0   1  22   9   345  5419   0   0   0   0  ...  255  1.00  0.00   
57449    0   1  45   5     0     0   0   0   0   0  ...    2  0.01  0.07   

          35    36   37   38   39   40  41  
317921  1.00  0.00  0.0  0.0  0.0  0.0  18  
171422  1.00  0.00  0.0  0.0  0.0  0.0  18  
312181  1.00  0.00  0.0  0.0  0.0  0.0  18  
87346   0.01  0.02  0.0  0.0  0.0  0.0  11  
57449   0.00  0.00  1.0  1.0  0.0  0.0  

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(98804, 42)
(98804,)
(395217, 42)
(395217,)


In [5]:
print(y_test.reshape(-1,1).shape)

(395217, 1)


In [6]:
#training

knn = KNeighborsClassifier(n_neighbors=3)
trainSet = np.array(X_train)
labelsSet = np.ravel(y_train)
trainSet = trainSet.astype(np.float64)
labelsSet = labelsSet.astype(np.float64)
knn.fit(trainSet, labelsSet)      

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [7]:
### processing
processing_start_time = time.monotonic()    
testSet = np.array(X_test)
testSet = testSet.astype(np.float64)
#testSet = testSet.reshape(1,-1)

result = knn.predict(testSet)
processing_end_time = time.monotonic()    
print("processing in: {} seconds".format(processing_end_time - processing_start_time))
print(result.shape)
#print(result)


processing in: 696.734999999986 seconds
(395217,)


In [8]:
## result comparasion
#result = result.reshape(-1,1)

#y_test = y_test.reshape(-1,1)
print(y_test.shape)
df = pd.DataFrame()
df["result"] = result
df['truth'] = y_test.astype(np.float64)
res =df.loc[~(df['result'] == df['truth'])]
print(res)
#df[['pred', 'truth']].assign(NEresult)


(395217,)
        result  truth
393        5.0   10.0
580        9.0   15.0
644       10.0   11.0
667       10.0   15.0
965       11.0    0.0
1669      11.0    9.0
1731      11.0   17.0
1798      20.0   17.0
2093      11.0   18.0
3335      11.0    9.0
3423       5.0   10.0
3467      21.0   11.0
4199       9.0   17.0
5173       9.0   15.0
5188      11.0    4.0
6517      18.0   11.0
6651      11.0   17.0
7552      21.0   11.0
7985       5.0   10.0
9758      22.0   11.0
9763      11.0   12.0
11436      5.0   10.0
12505      9.0   17.0
12991      5.0   10.0
13913     11.0   21.0
13982     15.0   11.0
15652      5.0   15.0
15880     10.0    5.0
15896     11.0    1.0
16537      5.0   11.0
...        ...    ...
380483    15.0   17.0
382490     9.0   15.0
382970    14.0   11.0
383325    11.0   12.0
383527    15.0   17.0
383917    11.0   15.0
384050    18.0   11.0
384344     9.0   15.0
384481     9.0   17.0
384867    14.0   11.0
385026     1.0   11.0
385479     9.0   17.0
385878    11.0    9.0


In [11]:

filename = time.strftime("%Y%m%d-%H%M%S") + '.csv'
df.to_csv("results/standardapp_"+filename, index=False)

In [13]:
print("error rate: {}%".format(len(res)/len(df)*100))

error rate: 0.175093682711017%
